In [1]:
import json
import numpy as np
import pandas as pd
import os

In [2]:
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
# pass the following cell to code to build prediction metrics
# for all available pickle files in predictions_summary/

In [4]:
predictions_pkl_list = os.listdir(path='/home/developer/gcp/cbidmltsf/database/predictions_summary/')
predictions_pkl_list.sort()

print('{} pickle files available in database/predictions_summary/'.format(len(predictions_pkl_list)))

200 pickle files available in database/predictions_summary/


In [5]:
# pass the following cell to code to build prediction metrics
# for given sets of pickle files in predictions_summary/

topologies = ['DMSLSTM']
infrastructures = ['TPU']
experiment_no_strings = ['005']
execution_no_strings = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09']
datasets = ['test']
predictions_pkl_list = list()
for topology in topologies:
    for infrastructure in infrastructures:
        for experiment_no_string in experiment_no_strings:
            for execution_no_string in execution_no_strings:
                for dataset in datasets:
                    predictions_pkl_list.append(
                        '{}_{}_{}_{}_on_{}_tfrecord.pkl'.format(
                            topology,
                            infrastructure,
                            experiment_no_string,
                            execution_no_string,
                            dataset))

In [6]:
# a Pandas dataframe to store all predictions summary items
df_columns=['model_id', 'execution', 'dataset',
            'count', 'mae', 'rmse', 'smape',
            'mae_vector', 'rmse_vector', 'smape_vector']
df = pd.DataFrame(columns=df_columns)

In [7]:
# collect prediction items from all available pickle files into a single dataframe
for predictions_pickle in predictions_pkl_list:
    buffer_df = pd.read_pickle('../database/predictions_summary/{}'.format(predictions_pickle))
    # append buffer to final dataframe
    df = df.append(buffer_df, ignore_index=True)  

In [8]:
# the default number of displayed rows for a dataframe is 60, expand it
pd.set_option("display.max_rows", 200)

In [9]:
df.head()

,model_id,execution,dataset,count,mae,rmse,smape,mae_vector,rmse_vector,smape_vector
0,ARTRFDC_TPU_000,0,test,1872,101.044830,125.422415,0.039948,"[47.8510985415206, 67.55012551332132, 80.19862...","[65.86299546355286, 90.75086411245374, 107.060...","[0.019057982109775017, 0.027025182390041053, 0..."
1,ARTRFDC_TPU_000,1,test,1872,110.309990,136.327681,0.043377,"[49.87253955286792, 72.11261064578325, 85.5869...","[70.5845094819496, 97.81143834029527, 114.9812...","[0.019672606282578733, 0.02864809898686916, 0...."
2,ARTRFDC_TPU_000,2,test,1872,100.571465,124.453318,0.039781,"[49.76701394105569, 70.28462714822884, 81.5607...","[68.74878122069825, 93.71048410590784, 108.443...","[0.019780438293549973, 0.028056132289439536, 0..."
3,ARTRFDC_TPU_000,3,test,1872,105.786972,130.371412,0.041518,"[46.788078242896965, 66.75235070122613, 79.100...","[64.11180784511782, 88.31195158504795, 104.283...","[0.01856074971051215, 0.026616715342791038, 0...."
4,ARTRFDC_TPU_000,4,test,1872,106.291191,131.088980,0.041883,"[46.91391160753038, 66.69246843126085, 79.3359...","[64.01878025850338, 88.2317226635982, 104.8267...","[0.018770852065884018, 0.02684926106242952, 0...."


In [15]:
len(df['smape_vector'][0])

24

In [53]:
# an empty dataframe to store SMAPE stats per model and dataset
smape_columns = ['model_id', 'executions', 'smape_min', 'smape_mean', 'smape_max', 'smape_std']

smape_df = pd.DataFrame(columns=smape_columns)
smape_df

In [54]:
# build a SMAPE dataframe using only the two best experiments for each model
model_ids = [
    'ARTRFDC_TPU_000',
    'DMSLSTM_TPU_006',
    'DMSLSTM_TPU_007',
    'EDSLSTM_TPU_011',
    'EDSLSTM_TPU_012'
]

In [55]:
for model_id in model_ids:

    # a row dataframe to calculate smape for a given model_id
    row_df = pd.DataFrame([[model_id,
                            df[df['model_id'] == model_id][['smape']].count()[0],
                            df[df['model_id'] == model_id][['smape']].min()[0],
                            df[df['model_id'] == model_id][['smape']].mean()[0],
                            df[df['model_id'] == model_id][['smape']].max()[0],
                            df[df['model_id'] == model_id][['smape']].std()[0]
                           ]],
                          columns=smape_columns)
    
    smape_df = smape_df.append(row_df, ignore_index=True)    

In [56]:
# best metrics
smape_df.style.highlight_min(color = 'lightgreen', axis = 0)

,model_id,executions,smape_min,smape_mean,smape_max,smape_std
0,ARTRFDC_TPU_000,10,0.039595,0.041526,0.044192,0.001538
1,DMSLSTM_TPU_006,10,0.033339,0.033966,0.034321,0.000312
2,DMSLSTM_TPU_007,10,0.034282,0.034689,0.035118,0.000246
3,EDSLSTM_TPU_011,10,0.031125,0.067021,0.382468,0.110838
4,EDSLSTM_TPU_012,10,0.034766,0.085978,0.404606,0.119617


In [57]:
# worst metrics
smape_df.style.highlight_max(color = 'yellow', axis = 0)

,model_id,executions,smape_min,smape_mean,smape_max,smape_std
0,ARTRFDC_TPU_000,10,0.039595,0.041526,0.044192,0.001538
1,DMSLSTM_TPU_006,10,0.033339,0.033966,0.034321,0.000312
2,DMSLSTM_TPU_007,10,0.034282,0.034689,0.035118,0.000246
3,EDSLSTM_TPU_011,10,0.031125,0.067021,0.382468,0.110838
4,EDSLSTM_TPU_012,10,0.034766,0.085978,0.404606,0.119617


In [ ]:
# ToDo: produce inferences for a given period only
# let's say, only 7, 14, or 21 days ahead (168, 336, 504 rows)

# for a fair comparison, train models up to a given date
# then predict the first predictable 24 hours (DMSLSTM and EDSLSTM are direct, ARTRFDC is iterative) 
# then use a iterative process to extend the forecast window to 168, 336, or 504
# then build statistics for inferences made on the exact same period for the three model architectures

In [13]:
# pass the grouped dataframe to a temporal structure
grouped = df.groupby(['model_id', 'dataset']).mean().reset_index()

In [20]:
grouped

,model_id,dataset,mae,rmse,smape
0,ARTRFDC_TPU_000,test,105.266441,130.245657,0.041526
1,ARTRFDC_TPU_001,test,114.289991,141.347790,0.045037
2,DMSLSTM_TPU_002,test,75.231964,95.524428,0.030646
3,DMSLSTM_TPU_003,test,75.039179,95.299967,0.030573
4,DMSLSTM_TPU_004,test,399.723941,441.360948,0.169398
5,DMSLSTM_TPU_005,test,75.008825,95.279031,0.030549
6,EDALSTM_TPU_000,test,109.808398,134.934060,0.043767
7,EDALSTM_TPU_001,test,153.253185,193.486667,0.062289
8,EDALSTM_TPU_002,test,103.648929,128.513852,0.041108
9,EDALSTM_TPU_003,test,137.647833,171.460254,0.055673


In [ ]:
# comment code that display vector metrics in summary, and display SMAPE only

In [ ]:
# an empty dataframe to store performance metrics values per model and dataset
metrics_columns = ['model_id', 'dataset', 'count',
                   'mae', 'rmse', 'smape',
                   'mae_vector', 'rmse_vector', 'smape_vector']

metrics_df = pd.DataFrame(columns=metrics_columns)

In [15]:
metrics_df

,model_id,dataset,count,mae,rmse,smape,mae_vector,rmse_vector,smape_vector


In [16]:
for _, row in grouped.iterrows():
    model_id, dataset = row['model_id'], row['dataset']
    # print (model_id, execution, dataset)
    # get a boolean to filter prediction_results_df on the model_id, execution, and dataset
    flag = \
    df.model_id.eq(model_id) & \
    df.dataset.eq(dataset)

    filtered_df = df[flag]
    
    # a row dataframe to calculate smape for a given model_id, counter, and dataset
    row_df = pd.DataFrame([[model_id,
                            dataset,
                            filtered_df.execution.count(),
                            filtered_df.mae.mean(),
                            filtered_df.rmse.mean(),
                            filtered_df.smape.mean(),
                            np.mean(np.array([row.mae_vector for _, row in filtered_df.iterrows()]), axis=0),
                            np.mean(np.array([row.rmse_vector for _, row in filtered_df.iterrows()]), axis=0),
                            np.mean(np.array([row.smape_vector for _, row in filtered_df.iterrows()]), axis=0)
                           ]],
                          columns=metrics_columns)
    
    metrics_df = metrics_df.append(row_df, ignore_index=True)

In [38]:
metrics_df

,model_id,dataset,count,mae,rmse,smape,mae_vector,rmse_vector,smape_vector
0,ARTRFDC_TPU_000,test,10,105.266441,130.245657,0.041526,"[48.15782056629148, 68.6241842090574, 81.06961...","[66.35716381745178, 91.39619700230463, 107.625...","[0.019161231985352446, 0.027469179361254405, 0..."
1,ARTRFDC_TPU_001,test,10,114.289991,141.347790,0.045037,"[53.37127647725945, 76.83904090946557, 90.3442...","[73.94806009422564, 102.71088098478621, 120.18...","[0.02115019082242984, 0.030583728634665647, 0...."
2,DMSLSTM_TPU_002,test,10,75.231964,95.524428,0.030646,"[47.50275246756417, 62.81771511350358, 70.3401...","[69.05000497436887, 88.91787445579465, 98.1215...","[0.019331143592711268, 0.025308525812985382, 0..."
3,DMSLSTM_TPU_003,test,10,75.039179,95.299967,0.030573,"[47.13030732018608, 61.62147447041103, 69.8876...","[68.39533640873195, 87.3328305368656, 97.58608...","[0.019245455032460862, 0.02485973775508655, 0...."
4,DMSLSTM_TPU_004,test,10,399.723941,441.360948,0.169398,"[367.3556483541216, 370.76828843525476, 375.82...","[406.04904058440303, 410.4573421797527, 417.03...","[0.1560966782463596, 0.15753657696416917, 0.15..."
5,DMSLSTM_TPU_005,test,10,75.008825,95.279031,0.030549,"[47.1619727407183, 61.57960617882865, 70.37438...","[68.66963054820776, 87.58226652641146, 98.0002...","[0.019203094441036402, 0.024799979186068227, 0..."
6,EDALSTM_TPU_000,test,10,109.808398,134.934060,0.043767,"[115.37159290928994, 108.34567482240739, 110.0...","[145.39138802638126, 138.3714061892139, 140.64...","[0.04620222546000671, 0.042965006422760875, 0...."
7,EDALSTM_TPU_001,test,10,153.253185,193.486667,0.062289,"[151.79511173617456, 155.29390578731415, 157.7...","[198.64089287719145, 202.4035752058279, 205.26...","[0.06187788563082449, 0.06320414108875394, 0.0..."
8,EDALSTM_TPU_002,test,10,103.648929,128.513852,0.041108,"[100.26792774200439, 102.33953087099137, 103.5...","[127.50221053981929, 130.04096466679425, 131.6...","[0.03996032252681926, 0.04082837710420212, 0.0..."
9,EDALSTM_TPU_003,test,10,137.647833,171.460254,0.055673,"[143.07213962779326, 143.5779073490816, 144.37...","[181.85520213849298, 183.00804107981048, 184.1...","[0.05837308124518027, 0.058224910997548854, 0...."


In [15]:
# DMSLSTM_TPU_002: unbalanced time-resolution-based [8, 8, 4] LSTM stacks, dense layer to vector output
# DMSLSTM_TPU_003: balanced time-resolution-based [8, 8, 8] LSTM stacks, dense layer to vector output
# DMSLSTM_TPU_004: balanced time-resolution-based [8, 8, 8] LSTM stacks, dense layer to time distributed
# DMSLSTM_TPU_005: balanced time-resolution-based [8, 8, 8] LSTM stacks, dense layer to vector output
# EDALSTM_TPU_000: m=256, h=256, 'relu' encoder-1/decoder-1, dense=[1]
# EDALSTM_TPU_001: m=256, h=256, 'relu' encoder-1/decoder-1, dense=[128, 32, 8, 1]
# EDALSTM_TPU_002: m=256, h=256, 'elu' encoder-1/decoder-1, batch_normalization, dense=[128, 32, 8, 1]
# EDALSTM_TPU_003: m=64, h=256, 'relu' encoder-1/decoder-1, batch_normalization, dense=[128, 32, 8, 1]
# EDALSTM_TPU_004: m=64, h=256, 'elu' encoder-1/decoder-1, batch_normalization [1]
# EDSLSTM_TPU_000: m=64, h=256, 'elu' encoder-3/decoder-1, batch_normalization, dense=[128, 32, 8, 1]
# EDSLSTM_TPU_001: m=64, h=256, 'elu' encoder-3/decoder-2, batch_normalization, dense=[1]
# EDSLSTM_TPU_008: m=64, h=256, 'elu' encoder-3/decoder-2, batch_normalization, dense=[1]
# EDSLSTM_TPU_009: m=64, h=256, 'elu' encoder-3/decoder-2, batch_normalization, dense=[1]

In [16]:
# reduce model set for analysis to the best experiment per architecture
# get detailed learning rate schedule for the models in the selected set

In [17]:
# DMSLSTM_TPU_005: balanced time-resolution-based [8, 8, 8] LSTM stacks, dense layer to vector output
# SMAPE = 0.030549
# wall_time = 26 s
# learning rate schedule
#    0-5: 0.0,
#    5-30: 0.01
#    30-60: 0.001
#    60-90: 0.0001
#    90-100: 0.00001

In [18]:
# EDALSTM_TPU_002: m=256, h=256, 'elu' encoder-1/decoder-1, batch_normalization, dense=[128, 32, 8, 1]
# SMAPE = 0.041108
# wall_time = 60 s
# learning rate schedule
#    0-2: 0.0,
#    2-10: 0.0001
#    10-15: 0.00005
#    15-20: 0.00002
#    20-40: 0.00001    

In [19]:
# EDSLSTM_TPU_008: m=64, h=256, 'elu' encoder-3/decoder-2, batch_normalization, dense=[1]
# SMAPE = 0.033614
# wall_time = 30 s
# learning rate schedule
#    0-1: 0.0
#    1-7: 0.004
#    7-14: 0.0004
#    14-20: 0.00004
#    20-22: 0.000004

In [20]:
# EDSLSTM_TPU_09: m=64, h=256, 'elu' encoder-3/decoder-2, batch_normalization, dense=[1]
# SMAPE = 0.034522
# wall_time = 30 s
# learning rate schedule
#    0-3: 0.0
#    3-15: 0.003
#    15-25: 0.0003
#    25-35: 0.00003
#    35-44: 0.000003

In [21]:
# use Bokeh to plot prediction statistics

In [22]:
plots = dict()

In [27]:
# pick a model and a dataset for predictions analysis
model_id = 'DMSLSTM_TPU_005' # balanced time-resolution-based LSTM stacks, dense layer to vector output
dataset = 'test'

In [28]:
# use a centipode-like command to get the index of the selected model/dataset in the metrics dataframe
index = metrics_df[metrics_df['model_id'] == model_id][metrics_df[metrics_df['model_id'] == model_id]['dataset'] == dataset].index

In [29]:
metric_string = 'smape_vector'

upper_label = metric_string.replace('_vector', '').upper()

plots[metric_string] = figure(
    plot_width=960,
    plot_height=400,
    title='{} values across forecasting horizon for {} on {} dataset'.format(upper_label,
                                                                             model_id,
                                                                             dataset))

plots[metric_string].grid.grid_line_alpha = 0.5

plots[metric_string].xaxis.axis_label = 'n-th forecasting step'
plots[metric_string].yaxis.axis_label = upper_label

plots[metric_string].line(np.arange(24),
                           metrics_df[metrics_df['model_id'] == model_id][metric_string][index[0]],
                           color='#A6CEE3',
                           legend_label=upper_label)

# uncomment the following two lines to save plot
# output_file('/home/developer/gcp/cbidmltsf/datasets/cfe/{}_H_kw.html'.format(device))
# save(fig_kw)

# uncomment the following line to display plot
show(plots[metric_string])

In [30]:
# pick a model and a dataset for predictions analysis
model_id = 'EDALSTM_TPU_002' # m=256, h=256, 'elu' encoder, 'elu' decoder, use_batch_normalization=True
dataset = 'test'

In [31]:
# use a centipode-like command to get the index of the selected model/dataset in the metrics dataframe
index = metrics_df[metrics_df['model_id'] == model_id][metrics_df[metrics_df['model_id'] == model_id]['dataset'] == dataset].index

In [32]:
metric_string = 'smape_vector'

upper_label = metric_string.replace('_vector', '').upper()

plots[metric_string] = figure(
    plot_width=960,
    plot_height=400,
    title='{} values across forecasting horizon for {} on {} dataset'.format(upper_label,
                                                                             model_id,
                                                                             dataset))

plots[metric_string].grid.grid_line_alpha = 0.5

plots[metric_string].xaxis.axis_label = 'n-th forecasting step'
plots[metric_string].yaxis.axis_label = upper_label

plots[metric_string].line(np.arange(24),
                           metrics_df[metrics_df['model_id'] == model_id][metric_string][index[0]],
                           color='#A6CEE3',
                           legend_label=upper_label)

# uncomment the following two lines to save plot
# output_file('/home/developer/gcp/cbidmltsf/datasets/cfe/{}_H_kw.html'.format(device))
# save(fig_kw)

# uncomment the following line to display plot
show(plots[metric_string])

In [33]:
# pick a model and a dataset for predictions analysis
model_id = 'EDSLSTM_TPU_008' # m=64, h=256, 'elu' encoder-3, 'elu' decoder-1, use_batch_normalization=True
dataset = 'test'

In [34]:
# use a centipode-like command to get the index of the selected model/dataset in the metrics dataframe
index = metrics_df[metrics_df['model_id'] == model_id][metrics_df[metrics_df['model_id'] == model_id]['dataset'] == dataset].index

In [35]:
metric_string = 'smape_vector'

upper_label = metric_string.replace('_vector', '').upper()

plots[metric_string] = figure(
    plot_width=960,
    plot_height=400,
    title='{} values across forecasting horizon for {} on {} dataset'.format(upper_label,
                                                                             model_id,
                                                                             dataset))

plots[metric_string].grid.grid_line_alpha = 0.5

plots[metric_string].xaxis.axis_label = 'n-th forecasting step'
plots[metric_string].yaxis.axis_label = upper_label

plots[metric_string].line(np.arange(24),
                           metrics_df[metrics_df['model_id'] == model_id][metric_string][index[0]],
                           color='#A6CEE3',
                           legend_label=upper_label)

# uncomment the following two lines to save plot
# output_file('/home/developer/gcp/cbidmltsf/datasets/cfe/{}_H_kw.html'.format(device))
# save(fig_kw)

# uncomment the following line to display plot
show(plots[metric_string])